In [12]:
import pandas as pd
import pytz
import numpy as np
import yfinance as yf
from datetime import datetime, timezone
import quantstats as qs
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.stats import kendalltau
from datetime import timedelta, date


In [2]:
# Set timeframe
start_date = "2008-01-01"
end_date = "2023-03-20"

# Define your current portfolio and target weights for each stock
current_portfolio = {'TIP': 0.10, #TIPS Bond
                     'ZB=F': 0.02, #20-30 Year Treasure Bond
                     'BND' : 0.03, #5-10 Intermediate Bond
                     'IEF' : 0.03, #7-10 Intermediate Bond
                     'JNK' : 0.05, #High-Yield Bond
                     'IGSB' : 0.05, #Short Term Corporation Bond
                     'AGG' : 0.02, #Aggregate Bond
                     'XLF' : 0.06, #Financial Select Sector
                     'XLU' : 0.03, #Utilities Select Sector
                     'XLP' : 0.03, #Consumer Staples Select Sector
                     'XLV' : 0.05, #Health Care Select Sector
                     'VGT' : 0.14, #Vanguard Information Technology
                     'XLK' : 0.05, #Technology Select Sector
                     'GLD' : 0.10, #Gold 
                     'SPYG' : 0.07, #SPY Growth
                     'QQQ' : 0.12, #Invesco QQQ 
                     'VNQ' : 0.05, #Real Estate Index Fund
                     'OXY' : 0.00, #Oil
                    }
                     
target_weights_earlymonth = {'TIP': 0.05, #TIPS Bond
                             'ZB=F': 0.01, #20-30 Year Treasure Bond
                             'BND' : 0.02, #5-10 Intermediate Bond
                             'IEF' : 0.02, #7-10 Intermediate Bond
                             'JNK' : 0.03, #High-Yield Bond
                             'IGSB' : 0.02, #Short Term Corporation Bond
                             'AGG' : 0.02, #Emerging Markets Bond
                             'XLF' : 0.08, #Financial Select Sector
                             'XLU' : 0.04, #Utilities Select Sector
                             'XLP' : 0.04, #Consumer Staples Select Sector
                             'XLV' : 0.08, #Health Care Select Sector
                             'VGT' : 0.12, #Vanguard Information Technology
                             'XLK' : 0.07, #Technology Select Sector
                             'GLD' : 0.11, #Gold
                             'SPYG' : 0.09, #SPY Growth 
                             'QQQ' : 0.12, #Invesco QQQ 
                             'VNQ' : 0.08, #Real Estate Index Fund
                             'OXY' : 0.00, #Oil
                            }

halloween_effect_weights = {'TIP': 0.06, #TIPS Bond
                             'ZB=F': 0.01, #20-30 Year Treasure Bond
                             'BND' : 0.02, #5-10 Intermediate Bond
                             'IEF' : 0.02, #7-10 Intermediate Bond
                             'JNK' : 0.02, #High-Yield Bond
                             'IGSB' : 0.02, #Short Term Corporation Bond
                             'AGG' : 0.01, #Emerging Markets Bond
                             'XLF' : 0.06, #Financial Select Sector
                             'XLU' : 0.03, #Utilities Select Sector
                             'XLP' : 0.03, #Consumer Staples Select Sector
                             'XLV' : 0.06, #Health Care Select Sector
                             'VGT' : 0.17, #Vanguard Information Technology
                             'XLK' : 0.06, #Technology Select Sector
                             'GLD' : 0.13, #Gold
                             'SPYG' : 0.10, #SPY Growth
                             'QQQ' : 0.14, #Invesco QQQ 
                             'VNQ' : 0.06, #Real Estate Index Fund
                             'OXY' : 0.00, #Oil
                           }

drawdown_weights = {'TIP': 0.40, #TIPS Bond
                     'ZB=F': 0.10, #20-30 Year Treasure Bond
                     'BND' : 0.15, #5-10 Intermediate Bond
                     'IEF' : 0.08, #7-10 Intermediate Bond
                     'JNK' : 0.05, #High-Yield Bond
                     'IGSB' : 0.05, #Short Term Corporation Bond
                     'AGG' : 0.08, #Emerging Markets Bond
                     'XLF' : 0.005, #Financial Select Sector
                     'XLU' : 0.005, #Utilities Select Sector
                     'XLP' : 0.01, #Consumer Staples Select Sector
                     'XLV' : 0.01, #Health Care Select Sector
                     'VGT' : 0.02, #Vanguard Information Technology
                     'XLK' : 0.01, #Technology Select Sector
                     'GLD' : 0.01, #Gold
                     'SPYG' : 0.00, #SPY Growth
                     'QQQ' : 0.01, #Invesco QQQ 
                     'VNQ' : 0.01, #Real Estate Index Fund
                     'OXY' : 0.00, #Oil
                   }

oil_weights = {'TIP': 0.10, #TIPS Bond
                 'ZB=F': 0.02, #20-30 Year Treasure Bond
                 'BND' : 0.03, #5-10 Intermediate Bond
                 'IEF' : 0.03, #7-10 Intermediate Bond
                 'JNK' : 0.03, #High-Yield Bond
                 'IGSB' : 0.02, #Short Term Corporation Bond
                 'AGG' : 0.02, #Emerging Markets Bond
                 'XLF' : 0.08, #Financial Select Sector
                 'XLU' : 0.04, #Utilities Select Sector
                 'XLP' : 0.04, #Consumer Staples Select Sector
                 'XLV' : 0.06, #Health Care Select Sector
                 'VGT' : 0.10, #Vanguard Information Technology
                 'XLK' : 0.06, #Technology Select Sector
                 'GLD' : 0.08, #Gold 
                 'SPYG' : 0.08, #SPY Growth
                 'QQQ' : 0.10, #Invesco QQQ 
                 'VNQ' : 0.06, #Real Estate Index Fund
                 'OXY' : 0.05, #Oil
                }

target_weights_oilearlymonth = {'TIP': 0.05, #TIPS Bond
                             'ZB=F': 0.01, #20-30 Year Treasure Bond
                             'BND' : 0.02, #5-10 Intermediate Bond
                             'IEF' : 0.02, #7-10 Intermediate Bond
                             'JNK' : 0.01, #High-Yield Bond
                             'IGSB' : 0.02, #Short Term Corporation Bond
                             'AGG' : 0.02, #Emerging Markets Bond
                             'XLF' : 0.08, #Financial Select Sector
                             'XLU' : 0.04, #Utilities Select Sector
                             'XLP' : 0.04, #Consumer Staples Select Sector
                             'XLV' : 0.08, #Health Care Select Sector
                             'VGT' : 0.10, #Vanguard Information Technology
                             'XLK' : 0.07, #Technology Select Sector
                             'GLD' : 0.08, #Gold
                             'SPYG' : 0.09, #SPY Growth 
                             'QQQ' : 0.12, #Invesco QQQ 
                             'VNQ' : 0.07, #Real Estate Index Fund
                             'OXY' : 0.08, #Exxon Oil
                            }

In [ ]:
# Define Functions
def isFirstWeek(row_datetime):
    dt = datetime.strptime(str(row_datetime), '%Y-%m-%d %H:%M:%S')
    return dt.day in range(1, 7)

def isHalloween(row_datetime):
    dt = datetime.strptime(str(row_datetime), '%Y-%m-%d %H:%M:%S')
    if dt.month in range(10,12):
        return True
    elif dt.month in range(1,3):
        return True
    else:
        return False

def isOil(row_datetime):
    dt = datetime.strptime(str(row_datetime), '%Y-%m-%d %H:%M:%S')
    if dt.month in range(5,7):
        return True
    else:
        return False

def earlyMonthWeights(df):
    index = 0
    for time in df.index:
        if isFirstWeek(time) == True and isOil(time) == True:
            for col in df:
                if isinstance(df[col][index], list):
                    df[col][index] = [df[col][index][0], df[col][index][1], target_weights_oilearlymonth[col]]
        elif isFirstWeek(time) == True and isOil(time) == False:
            for col in df:
                if isinstance(df[col][index], list):
                    df[col][index] = [df[col][index][0], df[col][index][1], target_weights_earlymonth[col]]
        index = index+1
    return df

def halloweenWeights(df):
    index = 0
    for time in df.index:
        if isHalloween(time) == True:
            for col in df:
                if isinstance(df[col][index], list):
                    df[col][index] = [df[col][index][0], df[col][index][1], halloween_effect_weights[col]]
        index = index+1
    return df

def oilWeights(df):
    index = 0
    for time in df.index:
        if isOil(time) == True:
            for col in df:
                if isinstance(df[col][index], list):
                    df[col][index] = [df[col][index][0], df[col][index][1], oil_weights[col]]
        index = index+1
    return df

def matchingDateIndividual(date1, date2):
    interval_length = timedelta(days=60)
    matching_dates = set()
    delta = date2 - date1
    if 0 <= delta.days <= interval_length.days:
        return True
    else:
        return False
    
def drawdownWeights(df, dates):
    index = 0
    for time in df.index:
        for ddPeriod in dates:
            if matchingDateIndividual(time, ddPeriod) == True:
                for col in df:
                    if isinstance(df[col][index], list):
                        df[col][index] = [df[col][index][0], df[col][index][1], drawdown_weights[col]]
        index = index+1
    return df

def calculateReturn(df, i, portfolio):
    df['Return'] = df['Close'].pct_change().fillna(0)
    df['Weight'] = portfolio[i]
    df['Combine'] = df[['Close', 'Return', 'Weight']].values.tolist()
    return df

def calculateReturnBenchmark(df):
    df['Return'] = df['Close'].pct_change().fillna(0)
    return df

def downloadData(download):
    x = yf.download(download, start=start_date, end=end_date)
    return x

def downloadAll(current_portfolio):
    dfCollection = {}
    for i in current_portfolio:
        dfCollection[i] = downloadData(i)
    return dfCollection

def createDataframePrice(portfolio, df):
    fund = pd.DataFrame(index=df[list(portfolio.items())[0][0]].index)
    count = 0
    for i in portfolio:
        if count == 0:
            fund[i] = df[i]['Close']
            count = 1
        else:
            fund = fund.join(df[i]['Close'], rsuffix=i)
    for i, col in zip(portfolio, fund.columns):
        fund = fund.rename(columns={col: str(i)})
    fund.fillna(0, inplace=True)
    return fund

def createDataframeCombine(portfolio, df):
    fund = pd.DataFrame(index=df[list(portfolio.items())[0][0]].index)
    count = 0
    for i in portfolio:
        if count == 0:
            fund[i] = df[i]['Combine']
            count = 1
        else:
            fund = fund.join(df[i]['Combine'], rsuffix=i)
    for i, col in zip(portfolio, fund.columns):
        fund = fund.rename(columns={col: str(i)})
    return fund

def calculateTotalReturn(df):
    row_sums = df.apply(lambda x: sum([i[1] * i[2] if isinstance(i, list) else 0 for i in x]), axis=1)
    return row_sums

def macroFactorModel(CPI, GDP, TB, UE):
    #Find changes in trend in time series data
    def trendChange(data, dataTotal):
        window_size = 5
        moving_avg = data.rolling(window_size).mean()
        diff = data - moving_avg
        threshold = 0.5
        trend_changes = []
        for i in range(1, len(diff)):
            if np.sign(diff[i]) != np.sign(diff[i-1]) and abs(diff[i]) > threshold:
                trend_changes.append(dataTotal.index[i])

        return trend_changes
    
    #Find Matching Dates between two arrays
    def matchingDate(date1s, date2s):
        interval_length = timedelta(days=60)
        matching_dates = set()
        for date1 in date1s:
            for date2 in date2s:
                delta = date2 - date1
                if 0 <= delta.days <= interval_length.days:
                    matching_dates.add(date1)
        return matching_dates
    
    CPI['DATE'] = pd.to_datetime(CPI['DATE'])
    CPI.set_index('DATE', inplace=True)
    GDP['DATE'] = pd.to_datetime(GDP['DATE'])
    GDP.set_index('DATE', inplace=True)
    TB['DATE'] = pd.to_datetime(TB['DATE'])
    TB.set_index('DATE', inplace=True)
    UE['DATE'] = pd.to_datetime(UE['DATE'])
    UE.set_index('DATE', inplace=True)
    
    #Merge Macro Indicators with SPY Dataset
    pCPI = pd.merge(CPI, SPYTest, left_index=True, right_index=True)
    pGDP = pd.merge(GDP, SPYTest, left_index=True, right_index=True)
    pTB = pd.merge(TB, SPYTest, left_index=True, right_index=True)
    pUE = pd.merge(UE, SPYTest, left_index=True, right_index=True)
    #Calculate Pearson Correlation
    corrCPI = np.corrcoef(pCPI['MEDCPIM158SFRBCLE'], pCPI['Close'])[0, 1]
    corrGDP = np.corrcoef(pGDP['GDP'], pGDP['Close'])[0, 1]
    corrTB = np.corrcoef(pTB['BOPGSTB'], pTB['Close'])[0, 1]
    corrUE = np.corrcoef(pUE['UNRATE'], pUE['Close'])[0, 1]
    #Calculate Tau Correlation
    tauCPI, p_valueCPI = kendalltau(pCPI['MEDCPIM158SFRBCLE'], pCPI['Close'])
    tauGDP, p_valueGDP = kendalltau(pGDP['GDP'], pGDP['Close'])
    tauTB, p_valueTB = kendalltau(pTB['BOPGSTB'], pTB['Close'])
    tauUE, p_valueUE = kendalltau(pUE['UNRATE'], pUE['Close'])
    
    #Calculate change in trends
    GDPChange = trendChange(pGDP['GDP'], pGDP)
    UEChange = trendChange(pUE['UNRATE'], pUE)
    TBChange = trendChange(pTB['BOPGSTB'], pTB)
    CPIChange = trendChange(pCPI['MEDCPIM158SFRBCLE'], pCPI)
    totalChange = [TBChange, CPIChange, UEChange, GDPChange]
    
    #Total Set of dates that overlap
    equivalence = []
    for i in range(len(totalChange)):
        for j in range(i+1, len(totalChange)):
            for x in matchingDate(totalChange[i], totalChange[j]):
                equivalence.append(x)
                
    drawdownDates = sorted(list(set(equivalence)))
    
    return drawdownDates, corrCPI, corrGDP, corrTB, corrUE, tauCPI, tauGDP, tauTB, tauUE

#Benchmark SPY
SPY = downloadData('SPY')
calculateReturnBenchmark(SPY)
benchmark = SPY['Return']
benchmark.index = benchmark.index.tz_localize(None)
SPYTest = SPY.copy()

#Get MacroIndicated Drawdown Periods
CPI = pd.read_csv('MEDCPIM.csv')
GDP = pd.read_csv('GDP.csv')
TB = pd.read_csv('TRADEBALANCE.csv')
UE = pd.read_csv('UNRATE.csv')
macroValues = macroFactorModel(CPI, GDP, TB, UE)   
drawdownDates = macroValues[0]
print('-------------------------------------------------------------------------------------------------')
print(drawdownDates)
print('-------------------------------------------------------------------------------------------------')

#Benchmark Ray Dalio AWF
allWeatherFund_portfolio = {'VTI': 0.30,
                            'TLT': 0.40,
                            'IEI': 0.15,
                            'GLD': 0.075,
                            'GSG': 0.074,
                           }
AWF = downloadAll(allWeatherFund_portfolio)
for i in allWeatherFund_portfolio:
    calculateReturn(AWF[i], i, allWeatherFund_portfolio)
awfPrice = createDataframePrice(allWeatherFund_portfolio, AWF)
awfCombine = createDataframeCombine(allWeatherFund_portfolio, AWF)
AWFReturn = calculateTotalReturn(awfCombine)

#Download Data and Calculate Return        
dfCollection = downloadAll(current_portfolio)
for i in current_portfolio:
    calculateReturn(dfCollection[i], i, current_portfolio)

#Create dataframe that holds [Price, Return] as values in each Column("Stock Name")
fundPrice = createDataframePrice(current_portfolio, dfCollection)
fundCombine = createDataframeCombine(current_portfolio, dfCollection)

#Assign early month weights
fundCombine = fundCombine.fillna(0)
fundCombine = halloweenWeights(fundCombine)
fundCombine = oilWeights(fundCombine)
fundCombine = earlyMonthWeights(fundCombine)
fundCombine = drawdownWeights(fundCombine, drawdownDates)

#Calculate total return
MF = calculateTotalReturn(fundCombine)
print('-------------------------------------------------------------------------------------------------')
print(MF)
print('-------------------------------------------------------------------------------------------------')

#QuantStat
qs.plots.snapshot(MF, figsize=(16, 8))
qs.reports.full(MF, benchmark, figsize=(16, 8))